In [1]:
%matplotlib inline

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
import json

In [2]:
fname = 'sp-pos-quot-dep-2021-03-28-19h20-reg.csv'
d = pd.read_csv(fname, 
                sep = ';', 
                parse_dates = ['jour'], 
                dtype = {'P': int, 'T': int},
                skiprows = [1])
d.sample(3)

,reg,jour,cl_age90,P,T,pop
40766,Pays de la Loire,2020-12-18,0,563,19834,3801797.0
48752,Île-de-France,2021-03-20,0,7824,74297,12278210.0
45328,Provence-Alpes-Côte d'Azur,2021-03-25,79,205,2694,509519.0


In [3]:
d.dtypes

reg                 object
jour        datetime64[ns]
cl_age90             int64
P                    int64
T                    int64
pop                float64
dtype: object

In [4]:
regs = d['reg'].unique()
ages = d['cl_age90'].unique()

In [5]:
d['incidence'] = d['P'] / d['pop'] * 100000
d['positivite'] = d['P'] / d['T'] * 100
d

In [7]:
tic1 = time.time()
d['incidence hebdo'] = np.zeros(len(d))
for reg in regs:
    for age in ages:
        d2 = d[(d['reg'] == reg) & (d['cl_age90'] == age)]
        d['temp'] = d2.apply(lambda x : (d2[ (d2['jour'] <= x['jour']) 
                                                        & (d2['jour'] > x['jour'] - np.timedelta64(1,'W'))]
                                                    ['incidence']
                                                    .sum()), axis = 1)
        d['incidence hebdo'] = d['incidence hebdo'] + d['temp'].fillna(0)
    toc1 = time.time()
    print('{:.2f} s : région {}'.format(-tic1+toc1, reg))

2.14 s : région Auvergne-Rhône-Alpes
4.29 s : région Bourgogne-Franche-Comté
6.39 s : région Bretagne
8.50 s : région Centre-Val de Loire
10.59 s : région Corse
12.70 s : région Grand Est
14.80 s : région Hauts-de-France
16.89 s : région Normandie
18.99 s : région Nouvelle-Aquitaine
21.08 s : région Occitanie
23.19 s : région Outre-mer
25.28 s : région Pays de la Loire
27.38 s : région Provence-Alpes-Côte d'Azur
29.47 s : région Île-de-France


In [8]:
d = d.drop(columns = ['temp'])
d.sample(3)

,reg,jour,cl_age90,P,T,pop,incidence,positivite,incidence hebdo
10067,Bretagne,2021-02-18,19,77,2198,417460.0,18.444881,3.503185,107.555215
33074,Occitanie,2020-10-13,79,143,1580,572470.0,24.979475,9.050633,128.391007
5161,Bourgogne-Franche-Comté,2020-10-12,19,123,1573,333965.0,36.830207,7.819453,135.044091


In [9]:
fname2 = fname[:-4] + '-processed.csv'
d.to_csv(fname2, sep = ';', index = False)